# Nettoyage des données

Le package `pyarrow` est nécessaire pour lire et écrire des fichiers parquet

In [11]:
%pip install pyarrow


[notice] A new release of pip is available: 23.3 -> 23.3.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
import pandas as pd

On charge le fichier de données

In [13]:
data_path = "data/IM2S_GHS_RSS_Bloc_2019_2022_anonyme.xlsx"
df = pd.read_excel(data_path)

On commence par renommer les colonnes pour plus de lisibilité

In [ ]:
df.rename(
    columns={
        'Date EntrÈe': 'date_entree',
        'Date Sortie': 'date_sortie',
        'Date Naissance': 'date_naissance',
        'Sexe': 'sexe',
        'CIM Diag Pr.': 'cim_diag_pr',
        'CCAM 1': 'ccam_1',
        'CCAM 2': 'ccam_2',
        'CCAM 3': 'ccam_3',
        'CCAM 4': 'ccam_4',
        'GHM Code': 'ghm_code',
        'Anesthesiste': 'anesthesiste',
        'Chirurgien': 'chirurgien',
        'AnnÈe': 'annee',
        'ID Patient': 'id_patient',
        'Date Inter': 'date_inter',
        'Nom Patient': 'nom_patient',
        "Heure entrée SSPI avant intervention (cela correspond à un SAS de pré-anesthésie). Si cette heure est la même que l'heure d'entrée en salle d'opération, cela signifie qu'il n'y as pas eu de passage en SSPI pré op": 'heure_entree_sspi',
        "Heure d'entrée en salle d'opération (calimed)": 'heure_entree_salle',
        'Heure Incision ': 'heure_incision',
        "Heure de sortie de salle d'opération (calimed)" : 'heure_sortie_salle',
        'Anesth Type' : 'anesth_type',
        'Anesth Loco_reg' : 'anesth_loco_reg',
        'DurÈe Sèjour en jour (1 pour ambu)' : 'duree_sejour',
        'Interv Type' : 'interv_type',
    },
    inplace=True
)

On supprime les colonnes et les lignes inutiles

In [ ]:
# Suppression des lignes non pertinentes
df = df.query("interv_type != 'Varices'")

# Suppression des lignes sans durée de séjour
df = df.query("duree_sejour < 1000")

# Suppression des colonnes inutiles
df = df.drop(
    columns=[
        'Cim Assoc 1',
        'Cim Assoc 2',
        'Cim Assoc 3',
        'Cim Assoc 4',
        'Cim Assoc 5',
        'Colonne2',
        'Colonne1',
    ]
)

On convertit les dates en datetime et on code les variables catégorielles

In [ ]:
# Conversion des dates en datetime
df['heure_entree_sspi'] = pd.to_timedelta(df['heure_entree_sspi'].astype(str))
df['heure_entree_salle'] = pd.to_timedelta(df['heure_entree_salle'].astype(str))
df['heure_incision'] = pd.to_timedelta(df['heure_incision'].astype(str))
df['heure_sortie_salle'] = pd.to_timedelta(df['heure_sortie_salle'].astype(str))

# Calcul de la durée d'opération
df['duree_sspi'] = df['heure_entree_salle'] - df['heure_entree_sspi']
df['duree_op'] = df['heure_sortie_salle'] - df['heure_entree_salle']

# Calcul de l'âge
df['age'] = df['date_entree'] - df['date_naissance']

# Ajout de la colonne urgence
df['urgence'] = df.cim_diag_pr.str.startswith('S')

# Codage des variables
df['cim_diag_pr'] = df['cim_diag_pr'].astype('category').cat.codes
df['ccam_1'] = df['ccam_1'].astype('category').cat.codes
df['ccam_2'] = df['ccam_2'].astype('category').cat.codes
df['ccam_3'] = df['ccam_3'].astype('category').cat.codes
df['ccam_4'] = df['ccam_4'].astype('category').cat.codes
df['ghm_code'] = df['ghm_code'].astype('category').cat.codes
df['anesthesiste'] = df['anesthesiste'].astype('category').cat.codes
df['chirurgien'] = df['chirurgien'].astype('category').cat.codes
df['anesth_type'] = df['anesth_type'].astype('category').cat.codes
df['anesth_loco_reg'] = df['anesth_loco_reg'].astype('category').cat.codes
df['interv_type'] = df['interv_type'].astype('category').cat.codes

On enregistre le résultat dans un fichier parquet

In [ ]:
df.to_parquet('data/clean_data.parquet', index=False)